In [530]:
import cv2
import os
from matplotlib import pyplot as plt
import numpy as np
import xml.etree.ElementTree as ET
from pathlib import Path
import pandas as pd
import difflib

dataDir = Path('../dataset/images') 
annotationsDir = Path('../dataset/annotations')

In [531]:
# Creating expected dataframes

def filelist(root, file_type):
    return [os.path.join(directory_path, f) for directory_path, directory_name, 
            files in os.walk(root) for f in files if f.endswith(file_type)]

def generate_train_multiple (anno_path): # Creates a dataframe with all images
    annotations = filelist(anno_path, '.xml')
    anno_list = []
    for anno_path in annotations:
        root = ET.parse(anno_path).getroot()
        anno = {}
        anno['filename'] = str(dataDir) + '/'+ root.find("./filename").text
        classArray = []
        for child in root:
            if child.tag == "object":
                for grandchild in child:
                    if grandchild.tag == "name":
                        if grandchild.text != "trafficlight":
                            classArray.append(grandchild.text)
        anno['expected'] = classArray
        if len(classArray) != 0:
            anno_list.append(anno)
            
    return pd.DataFrame(anno_list)

def generate_train_single (anno_path): # Creates a dataframe only with images that have a single roadsign to detect
    annotations = filelist(anno_path, '.xml')
    anno_list = []
    for anno_path in annotations:
        root = ET.parse(anno_path).getroot()
        anno = {}
        anno['filename'] = str(dataDir) + '/'+ root.find("./filename").text
        classArray = []
        for child in root:
            if child.tag == "object":
                for grandchild in child:
                    if grandchild.tag == "name":
                        if grandchild.text != "trafficlight":
                            classArray.append(grandchild.text)
        anno['expected'] = classArray
        if len(classArray) == 1:
            anno_list.append(anno)
            
    return pd.DataFrame(anno_list)

def generate_train_simple (anno_path): # Creates a dataframe with the first 170 images
    annotations = filelist(anno_path, '.xml')
    anno_list = []
    for anno_path in annotations:
        root = ET.parse(anno_path).getroot()
        anno = {}
        anno['filename'] = str(dataDir) + '/'+ root.find("./filename").text
        for child in root:
            if child.tag == "object":
                for grandchild in child:
                    if grandchild.tag == "name":
                        anno['expected'] = grandchild.text
        if int(anno['filename'][22:len(anno['filename'])-4]) < 170:
            anno_list.append(anno)
        
    return pd.DataFrame(anno_list)

In [532]:
# Image Pre-processing

# Improve Lighting

def improve_lighting(img):
    imgYUV = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)

    imgYUV[:, :, 0] = cv2.equalizeHist(imgYUV[:, :, 0])

    imgBetterLighting = cv2.cvtColor(imgYUV, cv2.COLOR_YUV2BGR)

    imgHSV = cv2.cvtColor(imgBetterLighting, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(imgHSV)

    lim = 255 - 50
    v[v > lim] = 255
    v[v <= lim] += 50

    final_hsv = cv2.merge((h, s, v))
    imgBetterLighting = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)

    return imgBetterLighting

# Smoothing

def image_smooth(img):
    imgWithMedianFilter = cv2.medianBlur(img, 5)

    return imgWithMedianFilter


In [533]:
# Image Segmentation by color

def image_segmentation(img):
    #set the bounds for the red hue
    lower_red_n1 = np.array([0,70,60])
    upper_red_n1 = np.array([10,255,255])

    lower_red_n2 = np.array([170,70,60])
    upper_red_n2 = np.array([180,255,255])

    lower_blue_n3 = np.array([78,158,124])
    upper_blue_n3 = np.array([138,255,255])

    #create a mask using the bounds set

    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)


    mask_1 = cv2.inRange(img_hsv, lower_red_n1, upper_red_n1)
    mask_2 = cv2.inRange(img_hsv, lower_red_n2, upper_red_n2)

    mask_red = mask_1 + mask_2

    mask_blue = cv2.inRange(img_hsv, lower_blue_n3, upper_blue_n3)

    return mask_blue, mask_red

In [534]:
# Image Thresholding and Morphological Operations

def morphological_ops(img):
    # Red

    # Removing Noise
    kernel = np.ones((3, 3),np.uint8)
    processed_red = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel, iterations = 1)
    processed_red = cv2.morphologyEx(processed_red, cv2.MORPH_DILATE, kernel, iterations = 1)

    # Floodfill

    red_floodfill = processed_red.copy()

    h, w = processed_red.shape[:2]

    mask = np.zeros((h+2, w+2), np.uint8)

    cv2.floodFill(red_floodfill, mask, (0,0), 255)

    red_floodfill_inv = cv2.bitwise_not(red_floodfill)

    filled_image = processed_red | red_floodfill_inv

    return filled_image

In [535]:
# Shape Recognition

def shape_recognition(img_red, img_blue, initial_image):
    
    result = ""
    results = []
    img_red_contours = initial_image
    img_blue_contours = initial_image

    # Red
    debug = "red"

    # Octagon Detection
    contours_red, hierarchy_red = cv2.findContours(img_red, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    maxArea = 0

    for cnt in contours_red:
        approx = cv2.approxPolyDP(cnt, 0.01*cv2.arcLength(cnt, True), True)

        if len(approx) == 8:
            cntArea = cv2.contourArea(cnt)
            if cntArea > maxArea:
                maxArea = cntArea
                maxContour = cnt
                result = "stop"

            results.append("stop")
            img_red_contours = cv2.drawContours(initial_image, [cnt], 0, (0,0,255), -1)

        elif len(approx) >= 12:
            cntArea = cv2.contourArea(cnt)
            if cntArea > maxArea:
                maxArea = cntArea
                maxContour = cnt
                result = "speedlimit"
         
            results.append("speedlimit")
            img_red_contours = cv2.drawContours(initial_image, [cnt], 0, (0,0,255), -1)

    # Blue

    contours_blue, hierarchy_blue = cv2.findContours(img_blue, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours_blue:
        approx = cv2.approxPolyDP(cnt, 0.01*cv2.arcLength(cnt, True), True)

        if len(approx) == 4:
            cntArea = cv2.contourArea(cnt)
            if cntArea > maxArea:
                maxArea = cntArea
                maxContour = cnt
                result = "crosswalk"
        
            results.append("crosswalk")
            # debug = "blue"
            img_blue_contours = cv2.drawContours(initial_image, [maxContour], 0, (255,0,0), -1)

    
    # if debug=="red":
        # plt.imshow(cv2.cvtColor(img_red_contours, cv2.COLOR_BGR2RGB))
        # plt.title('Foreground')
        # plt.axis('off')
        # plt.show()
    # elif debug == "blue":
        # plt.imshow(cv2.cvtColor(img_blue_contours, cv2.COLOR_BGR2RGB))
        # plt.title('Foreground')
        # plt.axis('off')
        # plt.show() 
    
    
    return result, results


In [536]:
# Detect the roadsigns in a certain image

def evaluate_image(imgPath):
    
    img = cv2.imread(imgPath)

    imgLighting = improve_lighting(img)
    imgSmooth = image_smooth(imgLighting)
    img_blue, img_red = image_segmentation(imgSmooth)
    processed_blue = morphological_ops(img_blue)
    processed_red = morphological_ops(img_red)
    result = shape_recognition(processed_red, processed_blue, img)
    
    return result

In [537]:
# Creating prediction Dataframe

def generate_prediction_multiple(df_compare): # Creates a dataframe with all the detected signs  
    pred_list = []                            # in the images from a given dataframe
    for index, row in df_compare.iterrows():
        pred = {}
        pred["filename"] = row["filename"]
        result, results = evaluate_image(row["filename"])
        pred["prediction"] = results
        pred_list.append(pred)
    return pd.DataFrame(pred_list)

def generate_prediction_single(df_compare): # Creates a dataframe with the single sign detected
    pred_list = []                          # in the images from a given dataframe
    for index, row in df_compare.iterrows():
        pred = {}
        pred["filename"] = row["filename"]
        result, results = evaluate_image(row["filename"])
        pred["prediction"] = result
        pred_list.append(pred)
    return pd.DataFrame(pred_list)

In [538]:
# Merging dataframes

def join_dataframes(df_train, df_pred): # Merges the dataframe with the expected results 
                                        # with the dataframe of the signs detected by our algorithm, 
                                        # adding a column that reflects the similarity between the two
    df_merged = pd.merge(df_train, df_pred, on='filename')

    for index, row in df_merged.iterrows():
        sm = difflib.SequenceMatcher(None,row["expected"],row["prediction"])
        df_merged.at[index,"Similarity"] = sm.ratio()

    return df_merged

def join_dataframes_simple(df_train, df_pred):  # Merges the dataframe with the expected result 
                                                # with the dataframe of the sign detected by our algorithm, 
                                                # adding a column that reflects the similarity between the two
    df_merged = pd.merge(df_train, df_pred, on='filename')

    for index, row in df_merged.iterrows():
        sm = difflib.SequenceMatcher(None,row["expected"],row["prediction"])
        if sm.ratio() == 0:
            df_merged.at[index,"Similarity"] = 1
        elif sm.ratio() == 1:
            df_merged.at[index,"Similarity"] = 1
        else:
            df_merged.at[index,"Similarity"] = 0

    return df_merged

In [539]:
# Performance evaluation - Simple Images

# Evaluates the performance of our sign detection algorithm within the first 170 images,
# that are well illuminated. Included the trafficlight images to demonstrate the non 
# detection of false positives

df_compare = generate_train_simple(annotationsDir)
df_pred = generate_prediction_single(df_compare)

df_simple = generate_train_simple(annotationsDir)
df_merged = join_dataframes_simple(df_simple, df_pred)
print("Performance for simple images: " + str(df_merged["Similarity"].mean()*100) + " %")
df_merged.head()

Performance for simple images: 53.529411764705884 %


,filename,expected,prediction,Similarity
0,../dataset/images/road18.png,trafficlight,,1.0
1,../dataset/images/road128.png,crosswalk,speedlimit,0.0
2,../dataset/images/road100.png,speedlimit,speedlimit,1.0
3,../dataset/images/road24.png,trafficlight,speedlimit,0.0
4,../dataset/images/road30.png,trafficlight,,1.0


In [540]:
# Performance evaluation - One sign Images

# Evaluates the performance of our sign detection algorithm for images with only 1 sign 
# expected to be detected in the image

df_compare = generate_train_single(annotationsDir)
df_pred = generate_prediction_multiple(df_compare)

df_single = generate_train_single(annotationsDir)
df_merged = join_dataframes(df_single, df_pred)
print("Performance for simple images: " + str(df_merged["Similarity"].mean()*100) + " %")
df_merged.head()

Performance for simple images: 27.418667204263198 %


,filename,expected,prediction,Similarity
0,../dataset/images/road289.png,[stop],"[speedlimit, crosswalk, crosswalk, crosswalk, ...",0.0
1,../dataset/images/road538.png,[speedlimit],[],0.0
2,../dataset/images/road510.png,[speedlimit],[speedlimit],1.0
3,../dataset/images/road276.png,[speedlimit],"[crosswalk, crosswalk]",0.0
4,../dataset/images/road262.png,[speedlimit],[],0.0


In [541]:
# Performance evaluation - Multiple sign Images

# Evaluates the performance of our sign detection algorithm for all the images in the dataset

df_compare = generate_train_multiple(annotationsDir)
df_pred = generate_prediction_multiple(df_compare)

df_multiple = generate_train_multiple(annotationsDir)
df_merged = join_dataframes(df_multiple, df_pred)
print("Performance for simple images: " + str(df_merged["Similarity"].mean()*100) + " %")
df_merged.head()

Performance for simple images: 28.13665654014769 %


,filename,expected,prediction,Similarity
0,../dataset/images/road712.png,"[speedlimit, speedlimit]","[speedlimit, stop, speedlimit, speedlimit]",0.666667
1,../dataset/images/road706.png,"[speedlimit, speedlimit]",[],0.000000
2,../dataset/images/road289.png,[stop],"[speedlimit, crosswalk, crosswalk, crosswalk, ...",0.000000
3,../dataset/images/road538.png,[speedlimit],[],0.000000
4,../dataset/images/road510.png,[speedlimit],[speedlimit],1.000000
